In [ ]:
import pandas as pd

def build_feature_dataset(
    input_paths: list[str],
    output_path: str,
    region: str,
    cols: list[str],
    freq: str = "30T",
    plot: bool = True
) -> tuple[pd.DataFrame, pd.Series]:
    """
    Loading Features 
    Args:
        input_paths:    List of Parquet file paths. Each file must load into a DataFrame
                        whose columns are a MultiIndex with levels [region, variable_name].
        output_path:    File‐path (including filename) where the feature report should be written.
        region:         The first‐level column key (region) to subset by after concatenation.
        cols:           A list of variable names (second‐level columns) to keep, once we subset to `region`.
        freq:           A Pandas offset alias (e.g. "30T", "15T", "1H") used to resample each DataFrame.
                        Default is "30T".
        plot:           If True, calls `generate_feature_report(...)` on the final feature set.

    Returns:
        feat:   A DataFrame of shape [n_samples × n_features], containing:
                • the time‐features (weekday, hour, month, etc.),
                • the chosen columns in `cols`,
                • and any newly added columns (forward‐filled) for modeling.
        tar:    A pd.Series named "Imbalance_Minus_Spot", aligned with `feat.index`, 
                containing the (imbalance_price − spot_price) at each timestamp.
    """
    # ----------------------------------------------------------------------------------
    # Helpers for timezone‐normalization + resampling
    # ----------------------------------------------------------------------------------
    def _load_and_resample_one(path: str, freq_rule: str) -> pd.DataFrame:
        """
        Loads one Parquet file into a DataFrame with a DateTimeIndex, normalizes
        its index to Asia/Tokyo, and resamples to `freq_rule` using .mean().
        """
        df = pd.read_parquet(path)

        # Ensure index is datetime:
        df = df.copy()
        df.index = pd.to_datetime(df.index)

        # If tz‐naive → assume it's already JST, so localize to Asia/Tokyo.
        # If tz‐aware (e.g. UTC or anything), convert to Asia/Tokyo.
        if df.index.tz is None:
            df.index = df.index.tz_localize("Asia/Tokyo")
        else:
            df.index = df.index.tz_convert("Asia/Tokyo")

        # Resample to the requested frequency, taking the mean of each
        # (e.g. if `freq_rule="30T"`, each 30‐minute block is averaged).
        df_resampled = df.resample(freq_rule).mean()
        return df_resampled

    # ----------------------------------------------------------------------------------
    # 1) Load + resample each input DataFrame; collect start/end times
    # ----------------------------------------------------------------------------------
    loaded_dfs = []
    start_times = []
    end_times = []

    for path in input_paths:
        df_resampled = _load_and_resample_one(path, freq)
        loaded_dfs.append(df_resampled)

        # Record the new index range
        start_times.append(df_resampled.index.min())
        end_times.append(df_resampled.index.max())

    if not loaded_dfs:
        raise ValueError("`input_paths` must contain at least one parquet file.")

    # ----------------------------------------------------------------------------------
    # 2) Find the common date‐range: [latest_start, earliest_end]
    # ----------------------------------------------------------------------------------
    latest_start = max(start_times)
    earliest_end = min(end_times)

    if latest_start >= earliest_end:
        raise ValueError(
            f"No overlapping time‐range found among the loaded files. "
            f"latest_start={latest_start}, earliest_end={earliest_end}"
        )

    # 3) Truncate each DataFrame to [latest_start : earliest_end]
    aligned_dfs = [
        df.loc[latest_start : earliest_end] for df in loaded_dfs
    ]

    # ----------------------------------------------------------------------------------
    # 4) Concatenate side‐by‐side (axis=1)
    # ----------------------------------------------------------------------------------
    # Since each df had columns = MultiIndex [region, variable], 
    # the concatenation keeps the same MultiIndex column structure.
    concatenated = pd.concat(aligned_dfs, axis=1)

    # ----------------------------------------------------------------------------------
    # 5) Subset by region (first level) and then by `cols` (second level)
    # ----------------------------------------------------------------------------------
    # This picks out one “slice” of the MultiIndex at level=0 == region.
    try:
        df_region = concatenated[region]
    except KeyError:
        raise KeyError(f"Region '{region}' not found in the concatenated columns.")

    # Now df_region’s columns are the second level only. We keep exactly `cols`.
    missing = [c for c in cols if c not in df_region.columns]
    if missing:
        raise KeyError(f"The following requested columns are not present for region {region}: {missing}")

    df_region = df_region[cols]

    # ----------------------------------------------------------------------------------
    # 6) Find the first+last index where BOTH spot & imbalance are non‐NaN
    # ----------------------------------------------------------------------------------
    imb_col = "pri_imb_down_%_kwh_jst_min30_a"
    spot_col = "pri_spot_jepx_%_kwh_jst_min30_a"

    # Ensure those two are in `cols` (or else we can’t form the target)
    if imb_col not in df_region.columns or spot_col not in df_region.columns:
        raise KeyError(
            f"Cannot find both target columns ('{imb_col}' and '{spot_col}') in df_region. "
            f"Got columns={list(df_region.columns)}"
        )

    # Build a mask where both are non‐NaN:
    both_valid = (
        df_region[imb_col].notna() &
        df_region[spot_col].notna()
    )
    # If there is no timestamp where both are valid, it's an error:
    if not both_valid.any():
        raise ValueError(
            f"No timestamp exists where both '{imb_col}' and '{spot_col}' are non‐NaN."
        )

    valid_times = df_region.index[both_valid]
    crop_start = valid_times.min()
    crop_end = valid_times.max()

    # Crop the DataFrame so that the first row has both non‐NaN, and the last row has both non‐NaN
    df_region = df_region.loc[crop_start : crop_end]

    # ----------------------------------------------------------------------------------
    # 7) Forward‐fill any remaining NaNs (limit=1)
    # ----------------------------------------------------------------------------------
    df_region = df_region.ffill(limit=1)

    # ----------------------------------------------------------------------------------
    # 8) Construct time‐features
    #    (Assumes you already have a function `construct_time_features(df)` defined elsewhere.)
    # ----------------------------------------------------------------------------------
    construct_time_features(df_region)

    # ----------------------------------------------------------------------------------
    # 9) Create the target series: "Imbalance_Minus_Spot"
    # ----------------------------------------------------------------------------------
    tar = df_region[imb_col] - df_region[spot_col]
    tar.name = "Imbalance_Minus_Spot"

    # ----------------------------------------------------------------------------------
    # 10) Optionally generate a feature report
    #    (Assumes you already have `generate_feature_report(...)` imported.)
    # ----------------------------------------------------------------------------------
    if plot:
        # name="Features" is arbitrary; you can change if you like
        generate_feature_report(
            features=df_region,
            target=tar,
            document_name=output_path,
            name="Features"
        )

    # ----------------------------------------------------------------------------------
    # 11) Return the final feature‐DataFrame and the target‐Series
    # ----------------------------------------------------------------------------------
    return df_region, tar

In [ ]:
import pandas as pd

def load_and_encode_imbalance_cs(
    imbalance_path: str,
    region: str,
    timestamp_col: str = "timestamp"
) -> pd.DataFrame:
    """
    Load the imbalance_cs_train Parquet, filter to the rows where `zone == region`,
    and then create eight one-hot (dummy) columns indicating which other zones share
    the same `wide_area_category` block code at each timestamp.

    Args:
        imbalance_path:   Path to the Parquet file containing imbalance_cs_train data.
                          It is assumed to have columns:
                            - timestamp_col  (DatetimeIndex)
                            - "zone"         (string: one of the nine region names)
                            - "wide_area_category" (int: block code)
                            - …any number of other features…
        region:           The name of the zone you want to keep (e.g. "tokyo", "kansai", etc.)
        timestamp_col:    The name of the timestamp column in the file. After loading,
                          this column will be converted to a DateTimeIndex. Default "timestamp".
                          If you actually have separate "date" + "period" columns, see note below.

    Returns:
        df_region:  A DataFrame indexed by timestamp (tz-aware if the file was),
                    containing:
                      • all original columns from the imbalance file for rows where zone=region
                        (EXCEPT "zone" and "wide_area_category", which we drop once we extract them),
                      • plus eight new columns of the form "is_same_block_<zone_name>" (int),
                        giving 1 if that other zone shares the same wide_area_category code at that time,
                        else 0.

        Example columns:
            [ ... other tokyo features ..., 
              is_same_block_hokkaido,
              is_same_block_tohoku,
              is_same_block_chubu,
              is_same_block_hokuriku,
              is_same_block_kansai,
              is_same_block_chugoku,
              is_same_block_shikoku,
              is_same_block_kyushu,
              is_same_block_okinawa
            ]
    """
    # ----------------------------------------
    # 1) Read the Parquet
    # ----------------------------------------
    df = pd.read_parquet(imbalance_path)

    # ----------------------------------------
    # 2) Parse/normalize the timestamp index
    #    (If your file truly has a single datetime column:)
    # ----------------------------------------
    if timestamp_col not in df.columns:
        # If instead your file has 'date' + 'period' (30-minute slot):
        # uncomment + adjust the following as needed:
        #
        # df["datetime_jst"] = (
        #     pd.to_datetime(df["date"].astype(str))
        #     + pd.to_timedelta((df["period"] - 1) * 30, unit="m")
        # )
        # df["datetime_jst"] = df["datetime_jst"].dt.tz_localize("Asia/Tokyo")
        # df = df.set_index("datetime_jst")
        # 
        # In that case, just reassign timestamp_col = "datetime_jst":
        # timestamp_col = "datetime_jst"
        #
        # For now, I’ll raise an error so you can correct to your actual schema:
        raise KeyError(
            f"Column '{timestamp_col}' not found in {imbalance_path}. "
            f"Either rename your datetime column to '{timestamp_col}', or supply "
            f"‘date’ + ‘period’ parsing logic above."
        )
    else:
        # If tz information is missing, you may need to localize → Asia/Tokyo.
        df[timestamp_col] = pd.to_datetime(df[timestamp_col])
        if df[timestamp_col].dt.tz is None:
            df[timestamp_col] = df[timestamp_col].dt.tz_localize("Asia/Tokyo")
        else:
            df[timestamp_col] = df[timestamp_col].dt.tz_convert("Asia/Tokyo")

        df = df.set_index(timestamp_col)

    # ----------------------------------------
    # 3) Pivot out the “wide_area_category” codes by zone
    #    so we can quickly see “at time t, zone X had code Y.”
    # ----------------------------------------
    # We only need “zone” and “wide_area_category” for this step.
    # If there are multiple rows for (timestamp, zone), you might want to
    # take the latest or drop duplicates first. Here, I’ll assume it’s unique.
    pivot_block = df[["zone", "wide_area_category"]].copy()
    # Make sure “zone” is a column, not the index:
    pivot_block = pivot_block.reset_index()  

    # Create a DataFrame whose index is timestamp, columns are the 9 zone names,
    # and values are the wide_area_category for that zone at that timestamp:
    block_df = pivot_block.pivot(
        index=timestamp_col,
        columns="zone",
        values="wide_area_category"
    )

    # ----------------------------------------
    # 4) Filter to just the “region” rows
    # ----------------------------------------
    # This gives us one row per timestamp for our region. If the original file
    # had multiple (timestamp, region) rows, you could .drop_duplicates(...) first.
    df_region = df[df["zone"] == region].copy()

    # If region never appears, we must error:
    if df_region.empty:
        raise KeyError(f"No rows found where zone == '{region}' in {imbalance_path}")

    # We’ll want to drop “zone” and “wide_area_category” from df_region once we extract them.
    # First, record the region’s block code (so we can compare to others):
    df_region["region_block_code"] = df_region["wide_area_category"]

    # ----------------------------------------
    # 5) Build the dummy columns
    # ----------------------------------------
    # For each timestamp t, block_df.loc[t] is a row whose columns are the 9 zone names,
    # and whose values are that zone’s wide_area_category code at time t.
    #
    # We want a boolean DataFrame: “is zone Z in the same block as our region at time t?”
    # That is: block_df.eq(region_block_code, axis=0).
    region_codes = df_region["region_block_code"].rename("region_block_code")

    # Align the index of block_df with the index of df_region (some timestamps might not match exactly)
    # We'll reindex block_df to only those timestamps where region appears.
    block_df_at_region_times = block_df.reindex(df_region.index)

    # Now compare: a True wherever block_df code == region_block_code
    same_block_bool = block_df_at_region_times.eq(region_codes, axis=0)

    # Convert True/False → 1/0
    same_block_int = same_block_bool.astype(int)

    # We do not need a dummy for the region itself (since it is obviously 1),
    # so drop that column if you like, or keep it. I’ll drop it to get exactly 8 columns:
    if region in same_block_int.columns:
        same_block_int = same_block_int.drop(columns=[region])

    # Rename the columns to “is_same_block_<zone>”
    same_block_int.columns = [f"is_same_block_{z}" for z in same_block_int.columns]

    # ----------------------------------------
    # 6) Merge these dummy columns back onto df_region
    # ----------------------------------------
    df_region = pd.concat([df_region, same_block_int], axis=1)

    # ----------------------------------------
    # 7) Drop the helper columns “zone” + “wide_area_category” + “region_block_code”
    #    (unless you want to keep them for reference)
    # ----------------------------------------
    df_region = df_region.drop(
        columns=["zone", "wide_area_category", "region_block_code"],
        errors="ignore"
    )

    # Now df_region has:
    #   • its original features (all columns except we dropped zone/wide_area_category),
    #   • plus exactly eight new columns “is_same_block_<other_zone>”.

    return df_region